### Plot TVC topography, AOI polygons and pit locations

In [ ]:
import rasterio
from rasterio.plot import show
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
%load_ext autoreload
%aimport aesop
%autoreload 1

Load topography, pits and aoi polygons:

In [ ]:
topography_rast = rasterio.open('../Data/topography/TPIsimple.tif')

In [ ]:
pits_with_topo = aesop.get_pit_topography()

In [ ]:
aoi_polygons = aesop.get_aoi_polygons()
aoi_polygons = aoi_polygons.to_crs(crs=topography_rast.crs.data)

In [ ]:
# Replace AOI labels
old_indices = aoi_polygons.index.values
new_indices = []

for idx in old_indices:
    new_indices.append(idx.replace('0','OI'))
aoi_polygons['label'] = new_indices

Plot figure 1:

In [ ]:
fig, ax = plt.subplots(figsize=(14,14))

cmap = ListedColormap(['#CD853F', '#FFFF66', '#90EE90', '#3399FF'])
rasterio.plot.show(topography_rast, ax=ax, cmap=cmap)

# pits_with_topo.plot(facecolor='#33FFFF', edgecolor='k', ax=ax, markersize=60, label='pits')
pits_with_topo.plot(marker='P', c='#33FFFF', ax=ax, markersize=100, label='Pits', edgecolor='k')

aoi_polygons.plot(edgecolor='#0066CC', facecolor='none', linewidth=2, ax=ax, label='Areas of Interest')
aoi_legend = Rectangle([0,0], 1,1, edgecolor='#0066CC', facecolor='none', linewidth=2, label='Areas of Interest')

ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

handles, labels = ax.get_legend_handles_labels()
legend_labels = {'#CD853F': 'Plateau', '#FFFF66': 'Valley', '#90EE90': 'Slopes', '#FFFFFF': ''}

patches = [Patch(color=color, label=label)
           for color, label in legend_labels.items()]

plt.legend(handles=patches+handles+[aoi_legend],
          bbox_to_anchor=(0.26, 1),
          facecolor='white',
          edgecolor='white',
          title='Topographic index:', fontsize=16)
plt.setp(ax.get_legend().get_title(), fontsize='16')

# Label AOIs
# xmin=0, ymin=1, xmax=2, ymax=[3] 
for idx, row in aoi_polygons.iterrows():
    if idx in ['A02', 'A04']:
        plt.annotate(text=row.label, xy=(row.geometry.bounds[0], row.geometry.bounds[3]),
                 horizontalalignment='center', fontsize=14)
    elif idx in ['A03']:
        plt.annotate(text=row.label, xy=(row.geometry.bounds[2], row.geometry.bounds[3]),
                 horizontalalignment='center', fontsize=14)
    elif idx in ['A05']:
        plt.annotate(text=row.label, xy=(row.geometry.bounds[0], row.geometry.bounds[1]),
                 horizontalalignment='left', verticalalignment='top',fontsize=14)
    else:
        plt.annotate(text=row.label, xy=(row.geometry.bounds[0], row.geometry.bounds[1]),
                 horizontalalignment='right', fontsize=14)

# Scale bar from https://geopandas.org/en/stable/gallery/matplotlib_scalebar.html        
ax.add_artist(ScaleBar(1))

plt.show()
plt.savefig("../Figs/Fig1.png",bbox_inches='tight')